# Gingr to QBO


Import the sheets - be sure to update the file path. 

<b> Todo </b> Have user select document from finder/explorer window

In [85]:
import pandas as pd
from datetime import datetime
import pandas.tseries.offsets as offsets

# Replace 'your_file.csv' with your CSV file path
df1 = pd.read_excel('/Users/antoniojimenez/coding/nocap/gingrtoqbo/gingrfile/original file copy-ek9.xlsx', sheet_name=0)
df2 = pd.read_excel('/Users/antoniojimenez/coding/nocap/gingrtoqbo/gingrfile/original file copy-ek9.xlsx', sheet_name=1)

# Display the first few rows to understand the structure
df1.head()

,Transaction ID,Location,Date,Owner,Exempt Charges,Taxable Charges,Tax Charged,Total Charged
0,5333.0,Everyday K9 Training,"Wed, 07/3, 11:20 am Heather Tyack",Sarah Smith,0,0.0,0.0,0
1,5339.0,Everyday K9 Training,"Sat, 08/17, 1:25 pm Heather Tyack",Holly Peterson,0,0.0,0.0,0
2,5363.0,Everyday K9 Training,"Fri, 07/5, 6:56 pm Heather Tyack",Fred Coble,360,0.0,0.0,360
3,5365.0,Everyday K9 Training,"Mon, 07/1, 4:10 pm Heather Tyack",Sasha Wasielewski,60,0.0,0.0,60
4,5366.0,Everyday K9 Training,"Mon, 07/1, 4:12 pm Heather Tyack",Sasha Wasielewski,0,0.0,0.0,0


In [86]:
df2.head()

,Payment ID,Invoice ID,Owner,Invoice Opened Date,Payment Date,Payment Method,Amount
0,4317: Heather Tyack,5365,Sasha Wasielewski,2024-07-01 16:10:00,2024-07-01 16:10:00,Cash,40
1,4320: Heather Tyack,5368,Jessica Frankel,2024-07-02 05:50:00,2024-07-02 05:50:00,Check,300
2,4321: Heather Tyack,5370,Matt Dixon,2024-07-02 20:14:00,2024-07-02 20:14:00,Cash,1200
3,4322: Heather Tyack,5375,Deb Volkman,2024-07-02 20:24:00,2024-07-02 20:24:00,Cash,90
4,4330: Heather Tyack,5378,Amy Ronshausen,2024-07-03 13:42:00,2024-07-03 13:42:00,Cash,240


### Get totals from Invoices and store them, then remove totals row

In [87]:
import pandas as pd

# Assuming df1 is already loaded from your Excel file
# Clean the column headers by stripping any leading or trailing whitespace
df1.columns = df1.columns.str.strip()

# Find the row in df1 that contains 'Totals' in the 'Owner' column
totals_row_df1 = df1[df1['Owner'].str.contains('Totals', case=False, na=False)]

# If a totals row exists, extract its values into variables
if not totals_row_df1.empty:
    totals_row = totals_row_df1.iloc[0]  # Get the first matching row
    
    # Assigning variables for each relevant column in the totals row
    exempt_charges = totals_row['Exempt Charges']
    taxable_charges = totals_row['Taxable Charges']
    tax_charged = totals_row['Tax Charged']
    total_charged = totals_row['Total Charged']
    
    # Print the variables to verify
    print("Exempt Charges:", exempt_charges)
    print("Taxable Charges:", taxable_charges)
    print("Tax Charged:", tax_charged)
    print("Total Charged:", total_charged)
else:
    print("No totals row found in df1.")

# Remove the row containing 'Totals' from df1
df1 = df1[~df1['Owner'].str.contains('Totals', case=False, na=False)]

# Display the modified df1
print("\nModified df1:")
print(df1.head())

Exempt Charges: 25060
Taxable Charges: 149.52
Tax Charged: 10.48
Total Charged: 25220

Modified df1:
   Transaction ID              Location                               Date  \
0          5333.0  Everyday K9 Training  Wed, 07/3, 11:20 am Heather Tyack   
1          5339.0  Everyday K9 Training  Sat, 08/17, 1:25 pm Heather Tyack   
2          5363.0  Everyday K9 Training   Fri, 07/5, 6:56 pm Heather Tyack   
3          5365.0  Everyday K9 Training   Mon, 07/1, 4:10 pm Heather Tyack   
4          5366.0  Everyday K9 Training   Mon, 07/1, 4:12 pm Heather Tyack   

               Owner  Exempt Charges  Taxable Charges  Tax Charged  \
0        Sarah Smith               0              0.0          0.0   
1     Holly Peterson               0              0.0          0.0   
2         Fred Coble             360              0.0          0.0   
3  Sasha Wasielewski              60              0.0          0.0   
4  Sasha Wasielewski               0              0.0          0.0   

   Total 

### Get totals from Payments and store total as variable

In [88]:
# Clean the column headers by stripping any leading or trailing whitespace
df2.columns = df2.columns.str.strip()

# Calculate the sum of the 'Amount' column
payment_totals = df2['Amount'].sum()

# Print the variable to verify
print("Payment Totals:", payment_totals)

Payment Totals: 26480


### Get year from Invoice Opened Date

In [89]:
# Extract the year from the 'Invoice Opened Date' column
# Convert the column to datetime if it's not already
df2['Invoice Opened Date'] = pd.to_datetime(df2['Invoice Opened Date'])

# Get the year from the first row of the 'Invoice Opened Date' column
year = df2['Invoice Opened Date'].iloc[0].year

# Display the year to verify
print("Extracted year:", year)

Extracted year: 2024


# Invoices

### Rename column headers and additional columns

In [90]:
# Rename column headers and additional columns
df1.columns = ['RefNumber', 'Location', 'TxnDate', 'Customer', 'LineAmount', 'Taxable Charges', 'TaxAmt', 'Total Charged']
df1['LineTaxable'] = None
df1['SalesTerm'] = None
df1['LineItem'] = None
df1.head()

,RefNumber,Location,TxnDate,Customer,LineAmount,Taxable Charges,TaxAmt,Total Charged,LineTaxable,SalesTerm,LineItem
0,5333.0,Everyday K9 Training,"Wed, 07/3, 11:20 am Heather Tyack",Sarah Smith,0,0.0,0.0,0,None,None,None
1,5339.0,Everyday K9 Training,"Sat, 08/17, 1:25 pm Heather Tyack",Holly Peterson,0,0.0,0.0,0,None,None,None
2,5363.0,Everyday K9 Training,"Fri, 07/5, 6:56 pm Heather Tyack",Fred Coble,360,0.0,0.0,360,None,None,None
3,5365.0,Everyday K9 Training,"Mon, 07/1, 4:10 pm Heather Tyack",Sasha Wasielewski,60,0.0,0.0,60,None,None,None
4,5366.0,Everyday K9 Training,"Mon, 07/1, 4:12 pm Heather Tyack",Sasha Wasielewski,0,0.0,0.0,0,None,None,None


### Clean up date and set format

In [91]:
# Extract the MM/DD part using regex and create a complete date with a year placeholder (2024 in this case)
df1['TxnDate'] = pd.to_datetime(df1['TxnDate'].str.extract(r'(\d{1,2}/\d{1,2})')[0] + f'/{year}', format='%m/%d/%Y')

# Convert the datetime object to the desired format M/DD/YYYY
df1['TxnDate'] = df1['TxnDate'].dt.strftime('%-m/%d/%Y')

# Display the modified DataFrame
df1.head()

,RefNumber,Location,TxnDate,Customer,LineAmount,Taxable Charges,TaxAmt,Total Charged,LineTaxable,SalesTerm,LineItem
0,5333.0,Everyday K9 Training,7/03/2024,Sarah Smith,0,0.0,0.0,0,None,None,None
1,5339.0,Everyday K9 Training,8/17/2024,Holly Peterson,0,0.0,0.0,0,None,None,None
2,5363.0,Everyday K9 Training,7/05/2024,Fred Coble,360,0.0,0.0,360,None,None,None
3,5365.0,Everyday K9 Training,7/01/2024,Sasha Wasielewski,60,0.0,0.0,60,None,None,None
4,5366.0,Everyday K9 Training,7/01/2024,Sasha Wasielewski,0,0.0,0.0,0,None,None,None


### Remove all lines with 0 in the 'Total Charged'

In [92]:
# Print the number of rows before removing rows with 0 in the 'Total Charged' column
initial_row_count = df1.shape[0]
print("Initial number of rows:", initial_row_count)

# Remove rows where 'Total Charged' is 0
df1 = df1[df1['Total Charged'] != 0]

# Print the number of rows after removal
final_row_count = df1.shape[0]
print("Number of rows after removal:", final_row_count)

Initial number of rows: 302
Number of rows after removal: 132


In [93]:
# Set SalesTerm to Due on Receipt for all columns.

In [94]:
# Insert "Due on Receipt" in the 'SalesTerm' column for all rows
df1['SalesTerm'] = "Due on Receipt"

# Verify the changes by displaying the first few rows
df1.head()

,RefNumber,Location,TxnDate,Customer,LineAmount,Taxable Charges,TaxAmt,Total Charged,LineTaxable,SalesTerm,LineItem
2,5363.0,Everyday K9 Training,7/05/2024,Fred Coble,360,0.0,0.0,360,None,Due on Receipt,None
3,5365.0,Everyday K9 Training,7/01/2024,Sasha Wasielewski,60,0.0,0.0,60,None,Due on Receipt,None
5,5368.0,Everyday K9 Training,7/02/2024,Jessica Frankel,300,0.0,0.0,300,None,Due on Receipt,None
7,5370.0,Everyday K9 Training,7/02/2024,Matt Dixon,1200,0.0,0.0,1200,None,Due on Receipt,None
12,5375.0,Everyday K9 Training,7/02/2024,Deb Volkman,90,0.0,0.0,90,None,Due on Receipt,None


In [95]:
# Ensure 'RefNumber' is treated as a string
df1['RefNumber'] = df1['RefNumber'].astype(int).astype(str)

df1.head()

,RefNumber,Location,TxnDate,Customer,LineAmount,Taxable Charges,TaxAmt,Total Charged,LineTaxable,SalesTerm,LineItem
2,5363,Everyday K9 Training,7/05/2024,Fred Coble,360,0.0,0.0,360,None,Due on Receipt,None
3,5365,Everyday K9 Training,7/01/2024,Sasha Wasielewski,60,0.0,0.0,60,None,Due on Receipt,None
5,5368,Everyday K9 Training,7/02/2024,Jessica Frankel,300,0.0,0.0,300,None,Due on Receipt,None
7,5370,Everyday K9 Training,7/02/2024,Matt Dixon,1200,0.0,0.0,1200,None,Due on Receipt,None
12,5375,Everyday K9 Training,7/02/2024,Deb Volkman,90,0.0,0.0,90,None,Due on Receipt,None


In [96]:
# Any line with both Exempt Charge and Taxable Charge, create two lines for both of them with same invoice/RefNumber. 
# - Make one line for LineAmount and update Col H total for that line
# - The other line has the taxable charge and tax amount

In [97]:
# Find rows with both 'LineAmount' (Exempt Charge) and 'Taxable Charges' present
condition = (df1['LineAmount'] != 0) & (df1['Taxable Charges'] != 0)
rows_to_split = df1[condition]

# List to hold the new rows
new_rows = []

# Iterate through the rows that need to be split
for index, row in rows_to_split.iterrows():
    # Create the first row for the exempt charge
    exempt_row = row.copy()
    exempt_row['Taxable Charges'] = 0
    exempt_row['TaxAmt'] = 0
    exempt_row['Total Charged'] = exempt_row['LineAmount']
    
    # Create the second row for the taxable charge, putting the taxable amount in 'LineAmount'
    taxable_row = row.copy()
    taxable_row['LineAmount'] = taxable_row['Taxable Charges']  # Move taxable charge to 'LineAmount'
    taxable_row['Taxable Charges'] = 0  # Set 'Taxable Charges' to 0
    taxable_row['Total Charged'] = taxable_row['LineAmount'] + taxable_row['TaxAmt']
    
    # Append the new rows to the list
    new_rows.append(exempt_row)
    new_rows.append(taxable_row)

# Create a DataFrame from the new rows
new_rows_df = pd.DataFrame(new_rows)

# Display the new_rows_df before appending to df1
new_rows_df

,RefNumber,Location,TxnDate,Customer,LineAmount,Taxable Charges,TaxAmt,Total Charged,LineTaxable,SalesTerm,LineItem
153,5536,Everyday K9 Training,8/10/2024,Cassie Hunt,300.00,0,0.00,300.0,None,Due on Receipt,None
153,5536,Everyday K9 Training,8/10/2024,Cassie Hunt,37.38,0,2.62,40.0,None,Due on Receipt,None
171,5555,Everyday K9 Training,8/15/2024,Brittney Capen,400.00,0,0.00,400.0,None,Due on Receipt,None
171,5555,Everyday K9 Training,8/15/2024,Brittney Capen,37.38,0,2.62,40.0,None,Due on Receipt,None
172,5556,Everyday K9 Training,8/15/2024,Brittany Carpenter,1200.00,0,0.00,1200.0,None,Due on Receipt,None
172,5556,Everyday K9 Training,8/15/2024,Brittany Carpenter,37.38,0,2.62,40.0,None,Due on Receipt,None


In [98]:
# Remove the original rows that were split from df1
df1 = df1.drop(rows_to_split.index)

# Append the new rows to the original df1
df1 = pd.concat([df1, new_rows_df], ignore_index=True)

# Sort the DataFrame by 'RefNumber' to ensure the split rows are consecutive
df1 = df1.sort_values(by='RefNumber').reset_index(drop=True)

# Display the modified and sorted DataFrame
print(df1)

    RefNumber              Location    TxnDate           Customer  LineAmount  \
0        5363  Everyday K9 Training  7/05/2024         Fred Coble       360.0   
1        5365  Everyday K9 Training  7/01/2024  Sasha Wasielewski        60.0   
2        5368  Everyday K9 Training  7/02/2024    Jessica Frankel       300.0   
3        5370  Everyday K9 Training  7/02/2024         Matt Dixon      1200.0   
4        5375  Everyday K9 Training  7/02/2024        Deb Volkman        90.0   
..        ...                   ...        ...                ...         ...   
130      5678  Everyday K9 Training  9/19/2024         Jason Hale        30.0   
131      5679  Everyday K9 Training  9/19/2024   Catalina Delgado        30.0   
132      5686  Everyday K9 Training  9/25/2024       Sarah Wilson        30.0   
133      5687  Everyday K9 Training  9/25/2024     Kristen Koenig        30.0   
134      5693  Everyday K9 Training  9/25/2024      Kimberly Wolk        40.0   

     Taxable Charges  TaxAm

### Move value of any thing In TaxableCharges to LineAmount column

In [99]:
# For any row with a value in the 'Taxable Charges' column, move that value to 'LineAmount' and set 'Taxable Charges' to 0
df1.loc[df1['Taxable Charges'] > 0, 'LineAmount'] += df1['Taxable Charges']
df1.loc[df1['Taxable Charges'] > 0, 'Taxable Charges'] = 0

# Display the modified DataFrame
print(df1)

    RefNumber              Location    TxnDate           Customer  LineAmount  \
0        5363  Everyday K9 Training  7/05/2024         Fred Coble       360.0   
1        5365  Everyday K9 Training  7/01/2024  Sasha Wasielewski        60.0   
2        5368  Everyday K9 Training  7/02/2024    Jessica Frankel       300.0   
3        5370  Everyday K9 Training  7/02/2024         Matt Dixon      1200.0   
4        5375  Everyday K9 Training  7/02/2024        Deb Volkman        90.0   
..        ...                   ...        ...                ...         ...   
130      5678  Everyday K9 Training  9/19/2024         Jason Hale        30.0   
131      5679  Everyday K9 Training  9/19/2024   Catalina Delgado        30.0   
132      5686  Everyday K9 Training  9/25/2024       Sarah Wilson        30.0   
133      5687  Everyday K9 Training  9/25/2024     Kristen Koenig        30.0   
134      5693  Everyday K9 Training  9/25/2024      Kimberly Wolk        40.0   

     Taxable Charges  TaxAm

In [100]:
# Fill in LineTaxable -  NON for lines with no Taxable Charges, TAX for those with Taxable Charges

In [101]:
# Loop through each row in the DataFrame and update the 'LineTaxable' column
for index, row in df1.iterrows():
    if row['TaxAmt'] > 0:
        df1.at[index, 'LineTaxable'] = 'TAX'
    else:
        df1.at[index, 'LineTaxable'] = 'NON'

# Display the modified DataFrame
print(df1)

    RefNumber              Location    TxnDate           Customer  LineAmount  \
0        5363  Everyday K9 Training  7/05/2024         Fred Coble       360.0   
1        5365  Everyday K9 Training  7/01/2024  Sasha Wasielewski        60.0   
2        5368  Everyday K9 Training  7/02/2024    Jessica Frankel       300.0   
3        5370  Everyday K9 Training  7/02/2024         Matt Dixon      1200.0   
4        5375  Everyday K9 Training  7/02/2024        Deb Volkman        90.0   
..        ...                   ...        ...                ...         ...   
130      5678  Everyday K9 Training  9/19/2024         Jason Hale        30.0   
131      5679  Everyday K9 Training  9/19/2024   Catalina Delgado        30.0   
132      5686  Everyday K9 Training  9/25/2024       Sarah Wilson        30.0   
133      5687  Everyday K9 Training  9/25/2024     Kristen Koenig        30.0   
134      5693  Everyday K9 Training  9/25/2024      Kimberly Wolk        40.0   

     Taxable Charges  TaxAm

In [102]:
# Fill in LineItem column: 'Sales' if NON-taxable or 'Sales of Product' if TAXable

In [103]:
# Loop through each row in the DataFrame and update the 'LineItem' column based on 'LineTaxable'
for index, row in df1.iterrows():
    if row['LineTaxable'] == 'NON':
        df1.at[index, 'LineItem'] = 'Sales'
    elif row['LineTaxable'] == 'TAX':
        df1.at[index, 'LineItem'] = 'Sales of Product'

# Display the modified DataFrame
print(df1)

    RefNumber              Location    TxnDate           Customer  LineAmount  \
0        5363  Everyday K9 Training  7/05/2024         Fred Coble       360.0   
1        5365  Everyday K9 Training  7/01/2024  Sasha Wasielewski        60.0   
2        5368  Everyday K9 Training  7/02/2024    Jessica Frankel       300.0   
3        5370  Everyday K9 Training  7/02/2024         Matt Dixon      1200.0   
4        5375  Everyday K9 Training  7/02/2024        Deb Volkman        90.0   
..        ...                   ...        ...                ...         ...   
130      5678  Everyday K9 Training  9/19/2024         Jason Hale        30.0   
131      5679  Everyday K9 Training  9/19/2024   Catalina Delgado        30.0   
132      5686  Everyday K9 Training  9/25/2024       Sarah Wilson        30.0   
133      5687  Everyday K9 Training  9/25/2024     Kristen Koenig        30.0   
134      5693  Everyday K9 Training  9/25/2024      Kimberly Wolk        40.0   

     Taxable Charges  TaxAm

In [104]:
# Verify totals from original excel files match updated totals

In [105]:
# Perform checks
line_amount_check = df1['LineAmount'].sum() == (exempt_charges + taxable_charges)
tax_amt_check = df1['TaxAmt'].sum() == tax_charged
total_charged_check = df1['Total Charged'].sum() == total_charged

# Print the results of the checks
print("Line Amount Check:", line_amount_check)
print("Tax Amount Check:", tax_amt_check)
print("Total Charged Check:", total_charged)

# Additional print statements to see actual values if checks fail
if not line_amount_check:
    print("Actual Line Amount Sum:", df1['LineAmount'].sum(), "Expected:", exempt_charges + taxable_charges)
if not tax_amt_check:
    print("Actual Tax Amount Sum:", df1['TaxAmt'].sum(), "Expected:", tax_charged)
if not total_charged_check:
    print("Actual Total Charged Sum:", df1['Total Charged'].sum(), "Expected:", total_charged)

Line Amount Check: True
Tax Amount Check: True
Total Charged Check: 25220


In [106]:
# Remove Location column

In [107]:
# Remove the 'Location' column
df1 = df1.drop(columns='Location')

# Payments

### Rename column headers and additional columns

In [108]:
# Rename column headers and additional columns
df2.columns = ['RefNumber', 'InvoiceApplyTo', 'Customer', 'Invoice Opened Date', 'TxnDate', 'PaymentMethod', 'LineAmount']
df2['DepositToAccount'] = None
df2.head()

,RefNumber,InvoiceApplyTo,Customer,Invoice Opened Date,TxnDate,PaymentMethod,LineAmount,DepositToAccount
0,4317: Heather Tyack,5365,Sasha Wasielewski,2024-07-01 16:10:00,2024-07-01 16:10:00,Cash,40,None
1,4320: Heather Tyack,5368,Jessica Frankel,2024-07-02 05:50:00,2024-07-02 05:50:00,Check,300,None
2,4321: Heather Tyack,5370,Matt Dixon,2024-07-02 20:14:00,2024-07-02 20:14:00,Cash,1200,None
3,4322: Heather Tyack,5375,Deb Volkman,2024-07-02 20:24:00,2024-07-02 20:24:00,Cash,90,None
4,4330: Heather Tyack,5378,Amy Ronshausen,2024-07-03 13:42:00,2024-07-03 13:42:00,Cash,240,None


### Remove ':' and name from RefNumber

In [109]:
# Strip the names and keep only the reference number
df2['RefNumber'] = df2['RefNumber'].str.split(':').str[0].str.strip()

# Display the updated df2
df2.head()

,RefNumber,InvoiceApplyTo,Customer,Invoice Opened Date,TxnDate,PaymentMethod,LineAmount,DepositToAccount
0,4317,5365,Sasha Wasielewski,2024-07-01 16:10:00,2024-07-01 16:10:00,Cash,40,None
1,4320,5368,Jessica Frankel,2024-07-02 05:50:00,2024-07-02 05:50:00,Check,300,None
2,4321,5370,Matt Dixon,2024-07-02 20:14:00,2024-07-02 20:14:00,Cash,1200,None
3,4322,5375,Deb Volkman,2024-07-02 20:24:00,2024-07-02 20:24:00,Cash,90,None
4,4330,5378,Amy Ronshausen,2024-07-03 13:42:00,2024-07-03 13:42:00,Cash,240,None


### Convert RefNumber and InvoiceApplyTo to string

In [110]:
# Convert 'RefNumber' and 'InvoiceApplyTo' columns to strings
df2['RefNumber'] = df2['RefNumber'].astype(str)
df2['InvoiceApplyTo'] = df2['InvoiceApplyTo'].astype(str)

# Display the updated df2
df2.dtypes

RefNumber                      object
InvoiceApplyTo                 object
Customer                       object
Invoice Opened Date    datetime64[ns]
TxnDate                datetime64[ns]
PaymentMethod                  object
LineAmount                      int64
DepositToAccount               object
dtype: object

### Convert Invoice Opened Date and TxnDate to M/DD/YYYY format

In [111]:
# Convert the columns to datetime format first
df2['Invoice Opened Date'] = pd.to_datetime(df2['Invoice Opened Date'])
df2['TxnDate'] = pd.to_datetime(df2['TxnDate'])

# Format the columns to 'M/DD/YYYY'
df2['Invoice Opened Date'] = df2['Invoice Opened Date'].dt.strftime('%-m/%d/%Y')
df2['TxnDate'] = df2['TxnDate'].dt.strftime('%-m/%d/%Y')

# Display the updated df2
df2.head()

,RefNumber,InvoiceApplyTo,Customer,Invoice Opened Date,TxnDate,PaymentMethod,LineAmount,DepositToAccount
0,4317,5365,Sasha Wasielewski,7/01/2024,7/01/2024,Cash,40,None
1,4320,5368,Jessica Frankel,7/02/2024,7/02/2024,Check,300,None
2,4321,5370,Matt Dixon,7/02/2024,7/02/2024,Cash,1200,None
3,4322,5375,Deb Volkman,7/02/2024,7/02/2024,Cash,90,None
4,4330,5378,Amy Ronshausen,7/03/2024,7/03/2024,Cash,240,None


In [112]:
### Assign Undeposited funds or Petty Cash to DepositToAccount column

In [113]:
# Update the 'DepositToAccount' column based on the conditions in 'PaymentMethod'
df2['DepositToAccount'] = df2['PaymentMethod'].apply(
    lambda x: 'Petty Cash' if x == 'Cash' else 
              'Undeposited funds' if x in ['Credit Card', 'Check'] else 
              'Other'
)

# Display the updated df2
df2.head()

,RefNumber,InvoiceApplyTo,Customer,Invoice Opened Date,TxnDate,PaymentMethod,LineAmount,DepositToAccount
0,4317,5365,Sasha Wasielewski,7/01/2024,7/01/2024,Cash,40,Petty Cash
1,4320,5368,Jessica Frankel,7/02/2024,7/02/2024,Check,300,Undeposited funds
2,4321,5370,Matt Dixon,7/02/2024,7/02/2024,Cash,1200,Petty Cash
3,4322,5375,Deb Volkman,7/02/2024,7/02/2024,Cash,90,Petty Cash
4,4330,5378,Amy Ronshausen,7/03/2024,7/03/2024,Cash,240,Petty Cash


## Save the updated Sheets to new excel file

In [114]:
# Convert 'TxnDate' in df1 and relevant date columns in df2 to datetime format
df1['TxnDate'] = pd.to_datetime(df1['TxnDate'])
df2['TxnDate'] = pd.to_datetime(df2['TxnDate'])

# Get the quarter and year from the first row of df1
first_date = df1['TxnDate'].iloc[0]
quarter = (first_date.month - 1) // 3 + 1
year = first_date.year

# Define the filename using the quarter and year
file_path = f'Q{quarter}-{year}-Invoices and Payments for Import.xlsx'

# Write both DataFrames to the same Excel file
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    # Write df1 to the first worksheet named 'Invoices'
    df1.to_excel(writer, sheet_name='Invoices', index=False)
    # Write df2 to the second worksheet named 'Payments'
    df2.to_excel(writer, sheet_name='Payments', index=False)
    
    # Access the workbook and worksheets
    workbook = writer.book
    worksheet_invoices = writer.sheets['Invoices']
    worksheet_payments = writer.sheets['Payments']

    # Define the currency format
    currency_format = workbook.add_format({'num_format': '$#,##0.00'})

    # Apply the currency format to specific columns in 'Invoices'
    worksheet_invoices.set_column('D:D', None, currency_format)  # LineAmount
    worksheet_invoices.set_column('E:E', None, currency_format)  # Taxable Charges
    worksheet_invoices.set_column('F:F', None, currency_format)  # TaxAmt
    worksheet_invoices.set_column('G:G', None, currency_format)  # Total Charged

    # Apply the currency format to the 'LineAmount' column in 'Payments'
    worksheet_payments.set_column('G:G', None, currency_format)  # LineAmount

print(f"File saved as: {file_path}")

File saved as: Q3-2024-Invoices and Payments for Import.xlsx
